In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, Dataset
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
# from sklearn.model_selection import train_test_split
from torchvision.transforms import transforms

from sklearn.neighbors import NearestNeighbors
import numpy as np
import random


# def get_augmentations():
#     return transforms.Compose([transforms.RandomHorizontalFlip(),
#                                transforms.RandomVerticalFlip(),
#                                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
#                                ])


class CustomTensorDataset(Dataset):
    """TensorDataset with support of transforms.
    Copied directly from https://stackoverflow.com/questions/55588201/pytorch-transforms-on-tensordataset
    """

    def __init__(self, tensors, transform=None):
        assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors)
        self.tensors = tensors
        self.transform = transform

    def __getitem__(self, index):
        x = self.tensors[0][index]

        if self.transform:
            x = self.transform(x)

        y = self.tensors[1][index]

        return x, y

    def __len__(self):
        return self.tensors[0].size(0)


def generate_synthetic(X, labels, n_neighbors=3):
    X = X.copy()
    print(X.shape)
    X_where_y0 = X[labels == 0]  # majority class
    X_where_y1 = X[labels == 1]
    X_where_y2 = X[labels == 2]
    y0_num = X_where_y0.shape[0]
    y1_num = X_where_y1.shape[0]
    y2_num = X_where_y2.shape[0]

    X_w_y1_reshaped = X_where_y1.reshape(X_where_y1.shape[0], -1)
    X_w_y2_reshaped = X_where_y2.reshape(X_where_y2.shape[0], -1)

    y1_upsample = y0_num - y1_num
    y2_upsample = y0_num - y2_num

    X_w_y1_synthetic = smote(X_w_y1_reshaped, y1_upsample, n_neighbors)
    X_w_y2_synthetic = smote(X_w_y2_reshaped, y2_upsample, n_neighbors)

    X_w_y1_synthetic = X_w_y1_synthetic.reshape(-1, *X_where_y1.shape[1:])
    X_w_y2_synthetic = X_w_y2_synthetic.reshape(-1, *X_where_y2.shape[1:])

    X_oversampled = np.vstack([X, X_w_y1_synthetic, X_w_y2_synthetic])
    y_oversampled = np.hstack([
        labels,
        np.ones(X_w_y1_synthetic.shape[0]),
        np.full(X_w_y2_synthetic.shape[0], 2)
    ])

    return X_oversampled, y_oversampled


def smote(X, num_oversamples, n_neighbors=5, seed=2109):
    np.random.seed(seed)
    n_samples, n_features = X.shape
    synthetic_samples = np.zeros((num_oversamples, n_features))

    nn = NearestNeighbors(n_neighbors=n_neighbors, algorithm='kd_tree')
    nn.fit(X)

    indices = np.random.randint(0, n_samples, size=num_oversamples)
    samples = X[indices]

    nnres = nn.kneighbors(samples, return_distance=False)

    nn_indices = nnres[np.arange(num_oversamples), np.random.randint(0, n_neighbors, size=num_oversamples)]
    nn_samples = X[nn_indices]

    diffs = nn_samples - samples
    synthetic_samples = samples + diffs * np.random.random(size=(num_oversamples, 1))

    return synthetic_samples.reshape(num_oversamples, *X.shape[1:])


def drop_nan_y(X, y):
    nan_indices = np.argwhere(np.isnan(y)).squeeze()
    mask = np.ones(y.shape, bool)
    mask[nan_indices] = False
    X = X[mask]
    y = y[mask]
    return X, y


def clean_x_data(X):
    X[np.isnan(X)] = np.nanmedian(X)
    X[X < 0] = 0
    X[X > 255] = 255
    # lower = np.percentile(X, 25) * 1.15
    # upper = np.percentile(X, 75) * 1.5
    # X[X < lower] = lower
    # X[X > upper] = upper
    return X

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)



# class CustomNeuralNetwork(nn.Module):
#     def __init__(self, input_size, classes=3, drop_prob=0.5):
#         super().__init__()
#         self.network = nn.Sequential(
#             nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#             nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#             nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#             nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#             nn.Flatten(),
#         )
#
#         self.fc = nn.Sequential(
#             nn.Linear(256, 128),
#             nn.ReLU(),
#             nn.Linear(128, 64),  # New fully connected layer
#             nn.ReLU(),
#             nn.Linear(64, 32),
#             nn.ReLU(),
#             nn.Linear(32, classes)
#         )
#
#
#     def forward(self, x):
#         x = self.network(x)
#         # print(x.shape)
#         x = self.fc(x)
#         return x


class CustomNeuralNetwork(nn.Module):
    def __init__(self, input_size, classes=3, drop_prob=0.3, seed=2109):
        super().__init__()
        torch.manual_seed(seed)
        self.network = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(),
            nn.Dropout(drop_prob),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.ReLU(),
            # nn.Dropout(drop_prob),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
        )

        self.fc = nn.Sequential(
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, classes)
        )

    def forward(self, x):
        x = self.network(x)
        # print(x.shape)
        x = self.fc(x)
        return x


class Model:
    """
    This class represents an AI model.
    """

    def __init__(self,
                 batch_size=20,
                 epochs=10,  # epochs seem to get worse after about 10 at num_components=256
                 # learning_rate=1e-3,
                 criterion=nn.CrossEntropyLoss,
                 num_components=256,
                 scaler=MinMaxScaler(),
                 learning_rate=0.0003826645125269827,
                 dropout=0.23535222860200122,
                 seed = 2109
                 ):
        """
        Constructor for Model class.

        Parameters
        ----------
        self : object
            The instance of the object passed by Python.
        """
        # TODO: Replace the following code with your own initialization code.
        # self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # self.device = torch.device("cpu")
        self.optimizer = None
        self.model = None
        self.batch_size = batch_size
        self.epochs = epochs
        self.learning_rate = learning_rate

        self.criterion = criterion()
        self.num_components = num_components
        self.pca = PCA(n_components=num_components, svd_solver='full')
        self.scaler = scaler
        self.dropout = dropout
        self.seed = seed

        self.g = torch.Generator()
        self.g.manual_seed(self.seed)


        torch.manual_seed(self.seed)
        np.random.seed(self.seed)
        random.seed(self.seed)

    def fit(self, X, y):
        """
        Train the model using the input data.

        Parameters
        ----------
        X : ndarray of shape (n_samples, channel, height, width)
            Training data.
        y : ndarray of shape (n_samples,)
            Target values.

        Returns
        -------
        self : object
            Returns an instance of the trained model.
        """
        # TODO: Add your training code.

        self.model = CustomNeuralNetwork(input_size=self.num_components, drop_prob=self.dropout)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)

        print('start')

        X, y = drop_nan_y(X, y)

        X = clean_x_data(X)

        # print("pre-synthetic")
        X, y = generate_synthetic(X, y, 5)
        # print(y.min())

        # X, X_test, y, y_test = train_test_split(X, y, test_size=100)
        # print(y.min())

        # Flatten and normalize the data
        flattened_data = X.reshape(X.shape[0], -1)

        normalized_data = self.scaler.fit_transform(flattened_data)
        # print("pre-pca")
        # print(y.min())
        pca_result = self.pca.fit_transform(normalized_data)
        reconstructed = self.pca.inverse_transform(pca_result)
        original_pca = reconstructed.reshape(-1, *X.shape[1:])

        pca_result_tensor = torch.tensor(original_pca, dtype=torch.float32)  #.to(self.device)
        labels_tensor = torch.tensor(y, dtype=torch.long)  # .to(self.device)

        # print(y.min())
        # dataset = CustomTensorDataset(tensors=(pca_result_tensor, labels_tensor), transform=get_augmentations())
        dataset = TensorDataset(pca_result_tensor, labels_tensor)
        train_loader = DataLoader(dataset=dataset, 
                                  batch_size=self.batch_size, 
                                  shuffle=True,
                                  num_workers=0,
                                    worker_init_fn=seed_worker,
                                    generator=self.g
                                  )
        # print("pre-epoch")

        epoch_losses = []
        for epoch in range(self.epochs):
            epoch_loss = 0
            # print(f"Epoch {epoch+1}")
            for inputs, labels in train_loader:
                # print(inputs, labels)
                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
                epoch_loss += loss.item()
            epoch_losses.append(epoch_loss / len(train_loader))
            print(f"Epoch {epoch + 1} loss: {epoch_losses[-1]}")

        return self

    def predict(self, X):
        """
        Use the trained model to make predictions.

        Parameters
        ----------
        X : ndarray of shape (n_samples, channel, height, width)
            Input data.

        Returns
        -------
        ndarray of shape (n_samples,)
        Predicted target values per element in X.

        """
        # TODO: Replace the following code with your own prediction code.
        X = clean_x_data(X)

        X = torch.from_numpy(X).float()
        # X.to(self.device)
        self.model.eval()

        flattened_data = X.reshape(X.shape[0], -1)
        normalized_data = self.scaler.transform(flattened_data)
        pca_result = self.pca.transform(normalized_data)
        reconstructed = self.pca.inverse_transform(pca_result)
        original_pca = reconstructed.reshape(-1, *X.shape[1:])

        print("fit shape:", pca_result.shape)

        original_pca = torch.tensor(original_pca, dtype=torch.float32)  #.to(self.device)
        with torch.no_grad():
            outputs = self.model(original_pca)
        return outputs.detach().numpy().argmax(axis=1)


In [3]:
%load_ext memory_profiler

In [4]:
%%time
%%memit


from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
import numpy as np


# Import packages
import pandas as pd
import numpy as np
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

# Load data
with open('data.npy', 'rb') as f:
    data = np.load(f, allow_pickle=True).item()
    X = data['image']
    y = data['label']


# Split train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

# Filter test data that contains no labels
# In Coursemology, the test data is guaranteed to have labels
nan_indices = np.argwhere(np.isnan(y_test)).squeeze()
mask = np.ones(y_test.shape, bool)
mask[nan_indices] = False
X_test = X_test[mask]
y_test = y_test[mask]

# Train and predict
model = Model()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Evaluate model predition
# Learn more: https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics
print("F1 Score (macro): {0:.2f}".format(f1_score(y_test, y_pred, average='macro'))) # You may encounter errors, you are expected to figure out what's the issue.

start
(2359, 3, 16, 16)
Epoch 1 loss: 0.6286987872454035
Epoch 2 loss: 0.2604705119049955
Epoch 3 loss: 0.21115822121846972
Epoch 4 loss: 0.1739596270151644
Epoch 5 loss: 0.15880519668384233
Epoch 6 loss: 0.13987965994275206
Epoch 7 loss: 0.12404102294910527
Epoch 8 loss: 0.11697164301013707
Epoch 9 loss: 0.10332488018338894
Epoch 10 loss: 0.09270679534502305
fit shape: (261, 256)
F1 Score (macro): 0.81
peak memory: 582.51 MiB, increment: 304.70 MiB
CPU times: total: 1min 19s
Wall time: 16.4 s


In [5]:
%%time
%%memit

# N fold cross validation
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

with open('data.npy', 'rb') as f:
    data = np.load(f, allow_pickle=True).item()
    X = data['image']
    y = data['label']


nan_indices = np.argwhere(np.isnan(y)).squeeze()
mask = np.ones(y.shape, bool)
mask[nan_indices] = False
X = X[mask]
y = y[mask]

num_folds = 10

model = Model()
kf = KFold(n_splits=num_folds, shuffle=True, random_state=2109)

f1_scores = []

for train_index, test_index in kf.split(X):
    model.fit(X=X[train_index], y=y[train_index])

    predictions = model.predict(X[test_index])

    score = f1_score(y[test_index], predictions, average='macro')

    f1_scores.append(score)
    print("f1:", score)

print("F1:", f1_scores)
print("Mean:", np.mean(f1_scores))
print("Std:", np.std(f1_scores))
print("Max:", np.max(f1_scores))
print("Min:", np.min(f1_scores))


start
(2358, 3, 16, 16)
Epoch 1 loss: 0.6426291365038466
Epoch 2 loss: 0.2630904563415198
Epoch 3 loss: 0.20912013877252186
Epoch 4 loss: 0.1727606677967641
Epoch 5 loss: 0.15446377038541767
Epoch 6 loss: 0.13701527513402664
Epoch 7 loss: 0.12748129640125244
Epoch 8 loss: 0.1111683818442678
Epoch 9 loss: 0.10759369346577628
Epoch 10 loss: 0.09330346542956698
fit shape: (262, 256)
f1: 0.5894590262050629
start
(2358, 3, 16, 16)
Epoch 1 loss: 0.6299949592176062
Epoch 2 loss: 0.2629056622083328
Epoch 3 loss: 0.21402566529440215
Epoch 4 loss: 0.18143489352363792
Epoch 5 loss: 0.1587032848847625
Epoch 6 loss: 0.14528984196414674
Epoch 7 loss: 0.12717498977442984
Epoch 8 loss: 0.11744619107672147
Epoch 9 loss: 0.10922619391822759
Epoch 10 loss: 0.0946389948573218
fit shape: (262, 256)
f1: 0.7086496156263598
start
(2358, 3, 16, 16)
Epoch 1 loss: 0.6128509369658612
Epoch 2 loss: 0.24906947081995084
Epoch 3 loss: 0.20886106601117566
Epoch 4 loss: 0.18678628938163028
Epoch 5 loss: 0.1639878808610

In [21]:
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from torch.utils.data import DataLoader, TensorDataset, Dataset
import numpy as np
import torch.nn as nn
from ray import train
from ray.air import session
from sklearn.model_selection import KFold

def drop_nan_y(X, y):
    nan_indices = np.argwhere(np.isnan(y)).squeeze()
    mask = np.ones(y.shape, bool)
    mask[nan_indices] = False
    X = X[mask]
    y = y[mask]
    return X, y

# Load data
with open('data.npy', 'rb') as f:
    data = np.load(f, allow_pickle=True).item()
    X = data['image']
    y = data['label']


X, y = drop_nan_y(X, y)

def train_test_model(config):
    model = Model(batch_size=config["batch_size"],
                    epochs=config["epochs"], 
                    # criterion=config["criterion"],
                    scaler=config["scaler"],
                    learning_rate=config["lr"])
    kf = KFold(n_splits=3)
    f1_scores = []


    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        f1 = f1_score(y_test, predictions, average='macro')
        f1_scores.append(f1)
    avg_f1 = np.mean(f1_scores)
    train.report({"score": avg_f1})
    
config = {
    "lr": tune.loguniform(1e-4, 1e-1),
    "drop_prob": tune.uniform(0.1, 0.5),
    "batch_size": tune.choice([10, 20, 30, 40, 50]),
    "epochs": tune.choice([10, 20, 30]),
    # "criterion": tune.choice([nn.CrossEntropyLoss, nn.MSELoss]),
    "scaler": tune.choice([MinMaxScaler(), StandardScaler()])
}

analysis = tune.run(
    train_test_model,
    config=config,
    num_samples=10,  # Number of times to sample from the hyperparameter space
    resources_per_trial={"cpu": 16, "gpu": 1}  # Adjust based on your system's resources

)
best_trial = analysis.get_best_trial("score","max","last")
print(best_trial.config)

2023-11-26 13:34:55,205	INFO tune.py:586 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


2023-11-26 13:35:02,219	WARNING tune.py:186 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2023-11-26 13:35:05,961	INFO tune.py:1047 -- Total run time: 10.76 seconds (6.97 seconds for the tuning loop).
2023-11-26 13:35:05,961	WARNING tune.py:1062 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: tune.run(..., resume=True)
2023-11-26 13:35:05,979	WARNING experiment_analysis.py:185 -- Failed to fetch metrics for 10 trial(s):
- train_test_model_87a01_00000: FileNotFoundError('Could not fetch metrics for train_test_model_87a01_00000: both result.json and progress.csv were not found at C:/Users/Ian/ray_results/train_test_model_2023-11-26_13-34-55/train_test_model_87a01_00000_0_batch_size=50,drop_prob=0.1490,epochs=10,lr=0.0029,scaler=ref_ph_908d9f2f_2023-11-26_13-34-55')
- train_test_model_87a01_00001: 

AttributeError: 'NoneType' object has no attribute 'config'

In [ ]:
# 2023-11-26 13:26:34,995	INFO tune.py:1047 -- Total run time: 583.14 seconds (583.05 seconds for the tuning loop).
# {'lr': 0.0003826645125269827, 'drop_prob': 0.23535222860200122, 'batch_size': 20, 'epochs': 10, 'scaler': StandardScaler()}

